<a href="https://colab.research.google.com/github/thitiwat05/229352-StatisticalLearning-or-Statistical-Learning-Labs./blob/main/Lab02_Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Statistical Learning for Data Science 2 (229352)
#### Instructor: Donlapark Ponnoprat

#### [Course website](https://donlapark.pages.dev/229352/)

## Lab #2

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve

# For Fashion-MNIST
from tensorflow.keras.datasets import fashion_mnist

# For 20 Newsgroups
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

## Part 1: Marketing Campaign Dataset - Manual Data Preprocessing & Logistic Regression

### Load the Marketing Campaign Dataset ([Data Information](https://archive.ics.uci.edu/dataset/222/bank+marketing))

The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be (`'yes'`) or not (`'no'`) subscribed.

In [3]:
bank_url = 'https://raw.githubusercontent.com/donlap/ds352-labs/main/bank.csv'

df = pd.read_csv(bank_url, sep=';', na_values=['unknown'])
df = df.drop(["emp.var.rate", "cons.price.idx", "cons.conf.idx",	"euribor3m", "nr.employed"], axis=1)
print("Shape of the dataset:", df.shape)
df.head()

Shape of the dataset: (41188, 16)


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,no
1,57,services,married,high.school,NaN,no,no,telephone,may,mon,149,1,999,0,nonexistent,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,no


### Data Exploration

In [4]:
print("--- Missing Values Count ---")
print(df.isnull().sum())

--- Missing Values Count ---
age               0
job             330
marital          80
education      1731
default        8597
housing         990
loan            990
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
y                 0
dtype: int64


In [5]:
print("--- Unique Values for Categorical Columns ---")
for col in df.select_dtypes(include='object').columns:
    print(f"\n'{col}' unique values:")
    print(df[col].value_counts(dropna=False)) # Include NaN counts

--- Unique Values for Categorical Columns ---

'job' unique values:
job
admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
NaN                330
Name: count, dtype: int64

'marital' unique values:
marital
married     24928
single      11568
divorced     4612
NaN            80
Name: count, dtype: int64

'education' unique values:
education
university.degree      12168
high.school             9515
basic.9y                6045
professional.course     5243
basic.4y                4176
basic.6y                2292
NaN                     1731
illiterate                18
Name: count, dtype: int64

'default' unique values:
default
no     32588
NaN     8597
yes        3
Name: count, dtype: int64

'housing' unique values:
housing
yes    21576
no     18622
NaN      990
Name: count, dtype: int64


### Data Preprocessing

In [9]:
# Map target variable 'y' to 0 (no) and 1 (yes)
df['y_new'] = df['y'].map({'yes':1, 'no':0})


# Drop 'duration' due to data leakage


# Define features (X) and target (y)
y = df['y_new']
X = df.drop(['y', 'y_new'], axis=1)
# Split the data BEFORE any transformations


# Print data shape



In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_train, X_test, y_train, y_test

(       age         job   marital            education default housing loan    contact month day_of_week  duration  campaign  pdays  previous     poutcome
 32177   31  management   married    university.degree      no     yes   no   cellular   may         fri       144         2    999         0  nonexistent
 22865   47  management   married    university.degree      no     yes  yes   cellular   aug         mon        57         2    999         0  nonexistent
 20012   45  management   married    university.degree      no     yes   no   cellular   aug         fri       334         4    999         0  nonexistent
 38991   36  technician    single  professional.course      no      no   no   cellular   dec         mon       371         1    999         0  nonexistent
 9054    42      admin.   married    university.degree     NaN     yes   no  telephone   jun         thu       160         2    999         0  nonexistent
 ...    ...         ...       ...                  ...     ...     ...

We will apply `StandardScaler()`, `OrdinalEncoder()`, and `OneHotEncoder()` on a few selected columns.

**1. Numerical Feature: `age` and `campaign` (Standard Scaling)**

In [12]:
num_cols_demo = ['age', 'campaign']

scaler = StandardScaler()

# Fit the scaler ONLY on the training data
X_train[num_cols_demo] = scaler.fit_transform(X_train[num_cols_demo])
X_test[num_cols_demo] = scaler.transform(X_test[num_cols_demo])

X_test

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome
24009,0.766980,admin.,married,high.school,no,no,no,telephone,oct,mon,312,-0.561910,999,0,nonexistent
8285,-0.770353,technician,married,professional.course,no,yes,no,telephone,jun,tue,220,-0.202404,999,0,nonexistent
4897,1.631729,management,married,basic.9y,no,no,no,telephone,may,wed,890,8.425739,999,0,nonexistent
20581,-0.770353,technician,married,professional.course,NaN,yes,no,cellular,aug,tue,796,0.516608,999,0,nonexistent
8501,-0.578186,entrepreneur,married,basic.4y,no,yes,no,telephone,jun,wed,250,-0.561910,999,0,nonexistent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13996,-0.674269,services,married,high.school,no,no,no,cellular,jul,fri,90,-0.202404,999,0,nonexistent
18128,-1.442936,services,single,basic.9y,no,yes,no,cellular,jul,wed,197,-0.561910,999,0,nonexistent
4904,-0.962519,admin.,single,high.school,no,no,no,telephone,may,wed,860,-0.202404,999,0,nonexistent
37031,-1.635102,student,single,high.school,no,no,no,cellular,jul,tue,370,-0.561910,999,1,failure


In [13]:
X_train.describe()

,age,duration,campaign,pdays,previous
count,2.883100e+04,28831.000000,2.883100e+04,28831.000000,28831.000000
mean,-1.478706e-18,259.733169,-4.436117e-18,962.733065,0.172696
std,1.000017e+00,260.832139,1.000017e+00,186.276443,0.493634
min,-2.211602e+00,0.000000,-5.619096e-01,0.000000,0.000000
25%,-7.703525e-01,103.000000,-5.619096e-01,999.000000,0.000000
50%,-1.938529e-01,180.000000,-2.024037e-01,999.000000,0.000000
75%,6.708966e-01,320.000000,1.571023e-01,999.000000,0.000000
max,5.571144e+00,4199.000000,1.453734e+01,999.000000,6.000000


**2. Ordinal Feature: `education` (Ordinal Encoding with Imputation)**

- **Imputation**

In [14]:
ord_col_demo = ['education']

imputer_ord = SimpleImputer(strategy='most_frequent')

## Write your code here
X_train[ord_col_demo] = imputer_ord.fit_transform(X_train[ord_col_demo])
X_test[ord_col_demo] = imputer_ord.transform(X_test[ord_col_demo])

X_train['education']

,education
32177,university.degree
22865,university.degree
20012,university.degree
38991,professional.course
9054,university.degree
...,...
29615,university.degree
24248,university.degree
13769,high.school
15970,high.school


- **Ordinal Encoding**

In [ ]:
education_categories = [
    'illiterate', 'basic.4y', 'basic.6y', 'basic.9y', 'high.school',
    'professional.course', 'university.degree', 'masters', 'doctorate'
]

In [17]:
# @title
ordinal_encoder = OrdinalEncoder(categories=[education_categories])

## Write your code here
X_train[ord_col_demo] = ordinal_encoder.fit_transform(X_train[ord_col_demo])
X_test[ord_col_demo] = ordinal_encoder.transform(X_test[ord_col_demo])

NameError: name 'education_categories' is not defined

In [16]:
X_train[ord_col_demo]

,education
32177,university.degree
22865,university.degree
20012,university.degree
38991,professional.course
9054,university.degree
...,...
29615,university.degree
24248,university.degree
13769,high.school
15970,high.school


**3. Nominal Feature: `job` (One-Hot Encoding with Imputation)**

- **Imputation**

In [24]:
nom_col_demo = ['job']

imputer_nom = SimpleImputer(strategy='most_frequent')
imputer_nom.fit(X_train[nom_col_demo])

X_train_imputed_nom_demo = imputer_nom.transform(X_train[nom_col_demo])
X_test_imputed_nom_demo = imputer_nom.transform(X_test[nom_col_demo])

KeyError: "None of [Index(['job'], dtype='object')] are in the [columns]"

- **Nominal Encoding**

In [18]:
onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

## Write your code here
X_train_onehot = onehot_encoder.fit_transform(X_train[['job']])
X_test_onehot = onehot_encoder.transform(X_test[['job']])

X_train_onehot.shape

(28831, 12)

In [23]:

X_train = pd.concat([X_train.reset_index(drop=True),
                     pd.DataFrame(X_train_onehot, columns=onehot_encoder.get_feature_names_out(['job']))], axis=1)
X_train = X_train.drop(['job'], axis=1) # ข้อมูลสุดท้ายที่ได้จะต้องเป็นตัวเลขเท่านั้น

X_train

KeyError: "['job'] not found in axis"

### **Exercise 1: Apply All Preprocessing & Train Logistic Regression**

Now, it's your turn to apply these preprocessing steps to *all* relevant columns and then train a Logistic Regression model.

**Instructions:**

1.  Look at the Variable Table in [this link](https://archive.ics.uci.edu/dataset/222/bank+marketing).
2. Make lists for `numerical_features`, `ordinal_features`, and `nominal_features`.
3. Preprocess the features. It is safer to make a copy of `X_train` using:
   ```
   X_train_copy = X_train.copy()
   X_test_copy = X_test.copy()
   ```
   and preprocess `X_train_copy` instead.

   **For nominal features, concat the one-hot encoded features using [`pd.concat(..., axis=1)`](https://pandas.pydata.org/docs/reference/api/pandas.concat.html) and drop the old nominal features from the dataframe.**
4. Train Logistic Regression on the preprocessed `X_train_copy` and `y_train`.
5. Evaluate the Model:
    *   Make predictions on the preprocessed `X_test_copy`.
    *   Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?


In [40]:
nominal_features = ['job','marital','contact','poutcome']
ordinal_features = ['education']
numerical_features = ['age','duration','campaign','pdays','previous',]

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_train_copy = X_train.copy()
X_test_copy = X_test.copy()

In [42]:
imputer_nom = SimpleImputer(strategy='most_frequent')
imputer_nom.fit(X_train_copy[nominal_features])
X_train_copy[nominal_features] = imputer_nom.fit_transform(X_train_copy[nominal_features])
X_test_copy[nominal_features] = imputer_nom.transform(X_test_copy[nominal_features])

In [43]:
onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

X_train_copy_onehot = onehot_encoder.fit_transform(X_train_copy[nominal_features])
X_test_copy_onehot = onehot_encoder.transform(X_test_copy[nominal_features])

X_train_copy = pd.concat([X_train_copy.reset_index(drop=True),
                     pd.DataFrame(X_train_copy_onehot, columns=onehot_encoder.get_feature_names_out(nominal_features))], axis=1)
X_train_copy = X_train_copy.drop(nominal_features, axis=1)

X_test_copy = pd.concat([X_test_copy.reset_index(drop=True),
                     pd.DataFrame(X_test_copy_onehot, columns=onehot_encoder.get_feature_names_out(nominal_features))], axis=1)
X_test_copy = X_test_copy.drop(nominal_features, axis=1)

In [45]:
imputer_ord = SimpleImputer(strategy='most_frequent')
X_train_copy[ord_col_demo] = imputer_ord.fit_transform(X_train_copy[ord_col_demo])
X_test_copy[ord_col_demo] = imputer_ord.transform(X_test_copy[ord_col_demo])

In [46]:
education_categories = [
    'illiterate', 'basic.4y', 'basic.6y', 'basic.9y', 'high.school',
    'professional.course', 'university.degree', 'masters', 'doctorate'
]

ordinal_encoder = OrdinalEncoder(categories=[education_categories])

X_train_copy[ord_col_demo] = ordinal_encoder.fit_transform(X_train_copy[ord_col_demo])
X_test_copy[ord_col_demo] = ordinal_encoder.transform(X_test_copy[ord_col_demo])

In [47]:
scaler = StandardScaler()

X_train_copy[numerical_features] = scaler.fit_transform(X_train_copy[numerical_features])
X_test_copy[numerical_features] = scaler.transform(X_test_copy[numerical_features])

In [48]:
X_train_copy = X_train_copy.drop(['default','housing','loan','month','day_of_week'], axis=1)
X_test_copy = X_test_copy.drop(['default','housing','loan','month','day_of_week'], axis=1)

In [49]:
model = LogisticRegression()
model.fit(X_train_copy, y_train)

y_pred = model.predict(X_test_copy)
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [50]:
target_names = ['non-subscribed a term deposit', 'subscribed a term deposit']
print(classification_report(y_test, y_pred, target_names=target_names))

                               precision    recall  f1-score   support

non-subscribed a term deposit       0.92      0.98      0.95     10936
    subscribed a term deposit       0.67      0.33      0.45      1421

                     accuracy                           0.90     12357
                    macro avg       0.79      0.66      0.70     12357
                 weighted avg       0.89      0.90      0.89     12357



## Part 2: Fashion-MNIST Dataset - Image Classification

### Load Fashion-MNIST Dataset

The Fashion-MNIST dataset consists of 28x28 grayscale images of fashion items.

In [26]:
(fm_X_train, fm_y_train), (fm_X_test, fm_y_test) = fashion_mnist.load_data()

print(f"Fashion-MNIST Train data shape: {fm_X_train.shape}")
print(f"Fashion-MNIST Train labels shape: {fm_y_train.shape}")
print(f"Fashion-MNIST Test data shape: {fm_X_test.shape}")
print(f"Fashion-MNIST Test labels shape: {fm_y_test.shape}")

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Fashion-MNIST Train data shape: (60000, 28, 28)
Fashion-MNIST Train labels shape: (60000,)
Fashion-MNIST Test data shape: (10000, 28, 28)
Fashion-MNIST Test labels shape: (10000,)


In [ ]:
print(f"First image {fm_X_train[0]}")
print(f"First label {fm_y_train[0]}")

### Visualize Fashion-MNIST Images

Let's see what these images look like.

In [ ]:
fashion_mnist_class_names = [
    'T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'
]

# Visualize the images
## Write your code here



### **Exercise 2: Preprocessing Images (Flatten and Scale)**

Images are 2D arrays (matrices of pixels) and pixel values are integers from 0-255. For Logistic Regression, we need:
*  **Flattening:** Convert each 28x28 image into a 1D array of 784 features.
*  **Scaling:** Normalize pixel values from [0, 255] to [0, 1].

**Instructions:**

1.   **Flatten:** Use the `.reshape()` method (see [documentation](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.reshape.html)). For `fm_X_train_binary` (shape `(num_samples, 28, 28)`), you want to reshape it to `(num_samples, 28*28)`.
2.  **Scale:** Divide the flattened pixel values by 255.0 to get values between 0 and 1.
3.   **Train Logistic Regression:**
    *   Initialize `LogisticRegression(solver='saga')`. `saga` is a good solver when both number of samples and number of features are large.
    *   Fit the model on your *processed* `fm_X_train_scaled` and `fm_y_train`.
4.   **Make Predictions:** Use `predict()` to make predictions on the *processed* `fm_X_test_scaled`.
5.   **Print Classification Report:** Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?
6.   **Visualize Misclassifications:**
    *   Find the indices in `fm_X_test_binary` where your model made incorrect predictions (i.e., `fm_y_pred != fm_y_test`).
    *   Select 5 of these misclassified images.
    *   Plot these images (using `plt.imshow`). For each image, print its true label and its predicted label.

In [ ]:
# --- YOUR CODE FOR EXERCISE 2 STARTS HERE ---





## Part 3: 20 Newsgroups Dataset - Text Classification

### Load 20 Newsgroups Dataset

The 20 newsgroups dataset comprises around 18000 newsgroups posts on 20 topics.

In [ ]:
news_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)
news_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=42)

X_train_news, y_train_news = news_train.data, news_train.target
X_test_news, y_test_news = news_test.data, news_test.target

print(f"Number of training documents: {len(X_train_news)}")
print(f"Number of test documents: {len(X_test_news)}")
print(f"Categories: {news_train.target_names}")

### Explore Sample Document

In [ ]:
# Print the first document and its class
## Write your code here



### Preprocessing: Text Vectorization Demonstration with `TfidfVectorizer`

$$
\text{TF-IDF}(t, d, D) = \text{TF}(t, d) \times \text{IDF}(t, D)
$$

Where:

$$
\text{TF}(t, d) = \frac{\text{number of word }t\text{ in } d}{\text{number of words in } d} \quad \text{ and } \quad
\text{IDF}(t, D) = \log\left(\frac{\text{total number of documents}}{\text{number of documents that contain word }t}\right).
$$

In [ ]:
sample_sentences = [
    "This is the first document.",
    "This document is the second document.",
    "And this is the third one.",
    "Is this the first document?"
]

vectorizer = TfidfVectorizer(stop_words='english')

# Fit and transform the sample sentences
sample_vec_output_sparse = # Write your code here

sample_vec_output_dense = sample_vec_output_sparse.toarray()

print(vectorizer.vocabulary_)
print(vectorizer.get_feature_names_out())
print(sample_vec_output_dense)

### **Exercise 3: Apply TF-IDF Vectorization to Full Dataset**

Now, apply `TfidfVectorizer` to the actual training and testing datasets for the 20 Newsgroups classification task.

**Instructions:**

1.  **Initialize `TfidfVectorizer`:**
    *   Initialize `TfidfVectorizer`. Use `stop_words='english'` to remove common words.
2.  **Fit and Transform Training Data:**
    *   Call `fit_transform()` on `X_train_news` to learn the vocabulary and transform the training text into TF-IDF features. Store the result in `X_train_vec`.
3.  **Transform Test Data:**
    *   Call `transform()` on `X_test_news` using the *already fitted* vectorizer. Store the result in `X_test_vec`. **Crucially, do not call `fit_transform()` on the test data!** This would cause data leakage.
4.  **Initialize Logistic Regression:**
    *   Initialize `LogisticRegression(solver='saga')`. `saga` is a good solver when both number of samples and number of features are large.
5.  **Train the Model:**
    *   Fit the model on your `X_train_vec` and `y_train_news`.
6.  **Make Predictions:**
    *   Make predictions using `predict()` on the `X_test_vec`.
7.  **Evaluate the Model:**
    *   Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?

In [ ]:
# --- YOUR CODE FOR EXERCISE 3 STARTS HERE ---


